In [2]:
import numpy as np
import os
import pathlib
import cv2
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential

In [3]:
# Define constants
IMAGE_SHAPE = (200, 200)  # Resize all images to this shape for MobileNetV2
BATCH_SIZE = 100
DATA_DIR = 'F:/indhu/Datasets/Medicinal plant dataset_30/'  # Update with your dataset path

In [4]:
# Load data
data_dir = pathlib.Path(DATA_DIR)
breeds = os.listdir(DATA_DIR)
dogs_images_dict = {}
dogs_labels_dict = {}
X, y = [], []
for index, category in enumerate(breeds):
    dogs_images_dict[category] = list(data_dir.glob(category + '/*'))
    dogs_labels_dict[category] = index

for breed_name, images in dogs_images_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized_img = cv2.resize(img, IMAGE_SHAPE)
        X.append(resized_img)
        y.append(dogs_labels_dict[breed_name])

X = np.array(X) / 255.0  # Normalize images
y = np.array(y)

In [5]:

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Create MobileNetV2 feature extractor
base_model_mobilenet = MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')
base_model_mobilenet.trainable = False  # Freeze the convolutional base

feature_extractor_mobilenet = Sequential([
    base_model_mobilenet,
    GlobalAveragePooling2D()
])

C:\Users\Indhuma\AppData\Local\Temp\ipykernel_8272\485066812.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model_mobilenet = MobileNetV2(input_shape=IMAGE_SHAPE + (3,), include_top=False, weights='imagenet')


In [7]:
# Extract features using MobileNetV2
X_train_features = feature_extractor_mobilenet.predict(X_train, batch_size=BATCH_SIZE, verbose=1)
X_test_features = feature_extractor_mobilenet.predict(X_test, batch_size=BATCH_SIZE, verbose=1)

36/36 ━━━━━━━━━━━━━━━━━━━━ 27s 690ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 686ms/step


In [8]:
# Initialize classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=200, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),
    "Support Vector Machine": SVC(kernel='rbf', probability=True, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Naive Bayes": GaussianNB()
}

In [9]:
# Evaluate classifiers using test set
for model_name, model in classifiers.items():
    print(f"Evaluating {model_name}...")
    model.fit(X_train_features, y_train)
    y_pred = model.predict(X_test_features)
    print(f"{model_name} Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(classification_report(y_test, y_pred))

Evaluating Random Forest...
Random Forest Accuracy: 0.8189
              precision    recall  f1-score   support

           0       0.89      0.96      0.93        26
           1       0.50      0.35      0.41        34
           2       0.91      1.00      0.96        32
           3       0.96      0.90      0.93        29
           4       0.81      0.76      0.79        29
           5       0.89      0.94      0.92        35
           6       0.94      0.89      0.92        19
           7       0.73      0.84      0.78        38
           8       0.94      0.89      0.92        38
           9       0.94      1.00      0.97        31
          10       0.78      0.81      0.79        36
          11       0.73      0.70      0.71        23
          12       0.86      0.66      0.75        29
          13       1.00      0.78      0.88        23
          14       0.88      0.96      0.92        24
          15       0.86      0.95      0.90        19
          16       0.8

In [10]:
# Evaluate classifiers using k-fold cross-validation
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for clf_name, clf in classifiers.items():
    print(f"K-Fold Cross Validation for {clf_name}...")
    precision_scores = []
    recall_scores = []
    f1_scores = []
    accuracy_scores = []

    for train_index, val_index in kf.split(X_train_features, y_train):
        X_train_fold, X_val_fold = X_train_features[train_index], X_train_features[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        clf.fit(X_train_fold, y_train_fold)
        y_val_pred = clf.predict(X_val_fold)

        accuracy_scores.append(accuracy_score(y_val_fold, y_val_pred))
        precision_scores.append(precision_score(y_val_fold, y_val_pred, average='weighted'))
        recall_scores.append(recall_score(y_val_fold, y_val_pred, average='weighted'))
        f1_scores.append(f1_score(y_val_fold, y_val_pred, average='weighted'))

    print(f"Average Accuracy: {np.mean(accuracy_scores):.4f}")
    print(f"Average Precision: {np.mean(precision_scores):.4f}")
    print(f"Average Recall: {np.mean(recall_scores):.4f}")
    print(f"Average F1 Score: {np.mean(f1_scores):.4f}\n")

K-Fold Cross Validation for Random Forest...
Average Accuracy: 0.8335
Average Precision: 0.8366
Average Recall: 0.8335
Average F1 Score: 0.8286

K-Fold Cross Validation for Logistic Regression...
Average Accuracy: 0.9258
Average Precision: 0.9286
Average Recall: 0.9258
Average F1 Score: 0.9252

K-Fold Cross Validation for K-Nearest Neighbors...
Average Accuracy: 0.8754
Average Precision: 0.8816
Average Recall: 0.8754
Average F1 Score: 0.8738

K-Fold Cross Validation for Support Vector Machine...
Average Accuracy: 0.9229
Average Precision: 0.9260
Average Recall: 0.9229
Average F1 Score: 0.9230

K-Fold Cross Validation for Decision Tree...
Average Accuracy: 0.4328
Average Precision: 0.4366
Average Recall: 0.4328
Average F1 Score: 0.4303

K-Fold Cross Validation for Naive Bayes...
Average Accuracy: 0.7607
Average Precision: 0.7798
Average Recall: 0.7607
Average F1 Score: 0.7641

